# Load the data

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
from main.AutoGPy import partition_data, SMCStructureLearning, plot_results_with_test, compute_gp_posterior
from sklearn.metrics import mean_squared_error

df = pd.read_csv("../data/iclaims.csv", header=0, names=["ds", "y"], parse_dates=["ds"])
df["y"] = np.log(df["y"])
df["t"] = (df["ds"] - df["ds"].min()).dt.days
x = df["t"].values
y = df["y"].values


# Split data into training and test sets
split_index = -20  # Last 20 time periods
x_train = x[:split_index]
y_train = y[:split_index]
x_test = x[split_index:]
y_test = y[split_index:]

# Partition training data
data_sequence = partition_data(x_train, y_train, num_partitions=5)



c:\Users\taylo\Documents\Work\study\test\AutoGPy\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Run SMC algo

In [2]:
# Initialize SMC
smc = SMCStructureLearning(
    num_particles=1000, num_rejuvenation_steps=3, max_kernel_depth=4
)

# Run SMC on training data
smc.run_smc(data_sequence)

# Collect particles and weights
particles = smc.particles
weights = smc.weights

# Find the particle with the highest weight
best_particle = particles[np.argmax(weights)]

print("Best Kernel Structure:")
print(best_particle.kernel)
print("Best Kernel Parameters:")
print(best_particle.kernel.get_params())

# Assign best_kernel
best_kernel = best_particle.kernel

# Plot the results with training and test data
plot_results_with_test(x_train, y_train, x_test, y_test, best_kernel)

# Evaluate on test data
mu_test, std_test = compute_gp_posterior(best_kernel, x_train, y_train, x_test)

# Compute MSE on test data
mse_test = mean_squared_error(y_test, mu_test)

print(f"Test MSE: {mse_test:.4f}")

Time step 1/5
Effective Sample Size (ESS): 1.00
Resampling particles...


AttributeError: module 'numpyro' has no attribute 'random'